In [1]:
import pandas as pd
import pathlib
cwd = pathlib.Path.cwd()
path_data=cwd/'mci1'
path_result=pathlib.Path(path_data/'result/')
path_result.mkdir(parents=True, exist_ok=True) 


In [2]:
#read data
df=pd.concat([pd.read_csv(path1,sep='\s+',header=None)
                   for path1 in path_data.rglob('m*')])
df.columns=['site','year','mon','day','var']
df['DateTime'] = pd.to_datetime(
        pd.to_datetime(
            df['year']*10000+df['mon']*100+df['day'],
            format='%Y%m%d'))
df=df.replace({-9990:999})
df['ci']=1
df['ci'][df['var']<=-6]=0
df['ci_x']=0
df['ci_x'][df['var']<=-6]=1
def find_end_index(index, index_list):
    diff = index_list.values-index
    return len(diff[diff>0])
df.head()


C:\Users\t\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\t\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,site,year,mon,day,var,DateTime,ci,ci_x
0,57469,1951,1,1,999,1951-01-01,1,0
1,57469,1951,1,2,999,1951-01-02,1,0
2,57469,1951,1,3,999,1951-01-03,1,0
3,57469,1951,1,4,999,1951-01-04,1,0
4,57469,1951,1,5,999,1951-01-05,1,0


In [ ]:
def find_end_index(index, index_list):
    diff = index_list.values-index
    return len(diff[diff>0])

In [ ]:
def q2(df1):

    df = df1.copy()
    df['sum'] = df['ci'].cumsum()
    df['sum_x'] = df['ci_x'].cumsum()
    count = df['sum'].value_counts(ascending=True)
    count_x = df['sum_x'].value_counts(ascending=True)
    # 开始时间序列
    df_start = pd.concat([df[df['sum'] == i].iloc[1] for i in count[count >
                                                                    10].index], axis=1).T.sort_values(by='DateTime', ascending=True)
    # 结束时间序列
    df_end = pd.concat([df[df['sum_x'] == i].iloc[0] for i in count_x[count_x >
                                                                       10].index], axis=1).T.sort_values(by='DateTime', ascending=True)
    # 匹配
    df_end_sle = pd.concat([df_end.iloc[-find_end_index(i, df_end.index)]
                            for i in df_start.index], axis=1).T
    # 计算ci和
    for i in range(len(df_start)):
        var = df.loc[df_start.index[i]:df_end_sle.index[i], :]['var'].sum()
        df_start.iloc[i]['var'] = df_end_sle.iloc[i]['var'] = var
    #输出
    df3 = pd.concat([df_start, df_end_sle, ]).sort_values(
        by='DateTime', ascending=True).drop(['DateTime', 'ci', 'ci_x', 'sum',
       'sum_x'],axis=1)
    df3.to_csv(
        (path_result/(str(df3.iloc[1]['site'])+'.txt')), index=None, sep=' ')
    return df3

result = df.groupby(['site']).apply(q2)

In [ ]:
result.head